# Ajuste del Pronóstico de Temperatura del WRF 1.5

### Nota:
Recuerde ajustar los links de las carpetas, directorios y url's que se encuentran a lo largo del código.

## Cargar Librerias
* Se cargan las librerías esenciales para la manipulación de datos, visualización, cálculos numéricos, manejo de fechas, interacción con la web y operaciones del sistema operativo.

In [383]:
import glob # Para buscar archivos en un directorio.
import pandas as pd # Para manipulación de datos.
import numpy as np # Para cálculos numéricos.
from datetime import datetime, date, timedelta # Para fechas y horas.
import requests # Para realizar solicitudes HTTP.
import urllib.request # Para trabajar con URLs.
import io # Para manejar entradas/salidas.
import os # Para interactuar con el sistema operativo.
import copy

## Configuración de Almacenamiento y Fechas para Datos de Temperatura
* Se proporciona las de rutas de carpetas ( `carpeta_modelo`, `carpeta_observado`) que se utilizarán para almacenar los datos del modelo y los datos observados de temperatura, respectivamente. Además, se establecen fechas de inicio (`fecha_inicio`) y fin (`fecha_fin`) para cargar archivos de temperatura en un rango específico de días.

* El diccionario `meses_espanol` asocia números de meses con sus nombres en español. Además, se definen enlaces base (`enlace_base_mod` y `enlace_base_obs`) que señalan a los servidores donde se encuentran los datos de temperatura pronosticada y observada.

In [384]:
# Ruta de la carpeta donde se almacenará los datos procesados
carpeta = r"C:/Users/arias/OneDrive/Documentos/UCR/TFG/Ajuste_Datos/Temperatura/Datos"

# Definición de la ruta de la carpeta donde se almacenarán los datos.
carpeta_modelo = f"{carpeta}/Modelo"
carpeta_observado = f"{carpeta}/Observado"

#Asignar rango de fechas para cargar archivos de temperatura
fecha_inicio = datetime.now().date() - timedelta(days=4)
fecha_fin = datetime.now().date() - timedelta(days=1)

# Diccionario que asocia números de meses con sus nombres en español
meses_espanol = {
    1: "enero", 2: "febrero", 3: "marzo", 4: "abril", 5: "mayo", 6: "junio", 7: "julio",
    8: "agosto", 9: "setiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
}

# Enlace base al servidor donde se encuentran los datos de temperatura pronosticada.
enlace_base_mod = "https://wrf1-5.imn.ac.cr/modelo/backup/TESIS/"

# Enlace base al servidor donde se encuentran los datos de temperatura observada.
enlace_base_obs = "http://intra-files.imn.ac.cr/Intranet_graficos/datos5minutos/tablas_registro"

In [385]:
fecha_inicio

datetime.date(2023, 12, 3)

In [386]:
fecha_fin

datetime.date(2023, 12, 6)

## Verificar que los archivos de Temperatura (Observado y Modelo) de los 3 días anteriores existan

### Función de Carga y Procesamiento de Datos de Temperatura

La función `cargar_datos` gestiona la carga y procesamiento de archivos relacionados con la temperatura. En primer lugar, utiliza la información de la fecha proporcionada, desglosándola en componentes como año, mes y día, así como el nombre del mes en español. Luego, construye los enlaces completos para los archivos de temperatura observada y pronosticada, utilizando estos componentes y las direcciones base proporcionadas.

La función realiza verificaciones para asegurarse de que los archivos de temperatura observada y pronosticada estén disponibles mediante solicitudes HTTP a los respectivos enlaces. Si ambos archivos están presentes, la función decodifica y lee el archivo CSV de temperatura pronosticada, verificando la presencia de valores negativos o cadenas en los datos. Si se encuentran tales valores problemáticos, se emite un mensaje y el archivo no se carga. En caso contrario, se almacenan los datos observados, los datos pronosticados y la fecha correspondiente en listas específicas.

En situaciones donde uno o ambos archivos no están disponibles, la función emite un mensaje indicando archivos faltantes y agrega la fecha a una lista de datos faltantes. Finalmente, la función devuelve las listas actualizadas de datos pronosticados, observados y las fechas de los archivos procesados. 

In [387]:
# Función para cargar datos y procesar archivos
def cargar_datos(fecha, fecha_anterior, COLUMN_NAMES, enlace_base_obs, enlace_base_mod, meses_espanol):
    
    # Desglosar la fecha en sus componentes: año, mes, día y nombre del mes en español
    año, mes, dia, mes_text = fecha.year, fecha.month, fecha.day, meses_espanol[fecha.month]

    # Crear el URL completo del archivo de temperatura observada utilizando los componentes de la fecha
    url_observado = f"{enlace_base_obs}/{año}/{mes_text}/temperaturas_{dia}_{mes}_{año}.txt"

    # Formar el enlace completo para el archivo CSV con la fecha actual y siguiente
    url_modelo = f"{enlace_base_mod}{fecha_anterior.strftime('%Y%m%d')}/{fecha.strftime('%Y%m%d')}.temperatura.csv"

    # Verificar si el archivo de temperatura observada existe antes de descargarlo
    response_obs = requests.get(url_observado)
    response_mod = requests.get(url_modelo)

    if response_obs.status_code == 200 and response_mod.status_code == 200:
        # Decodificar y leer los datos del archivo CSV de temperatura pronosticada
        mod_data = response_mod.content.decode('utf-8')
        df = pd.read_csv(io.StringIO(mod_data), header=None, names=COLUMN_NAMES)

        # Verificar la presencia de valores negativos o cadenas en el archivo de temperatura pronosticada
        negative_values = (df[COLUMN_NAMES[1:]] < 0).any(axis=0)
        str_values = df[COLUMN_NAMES[1:]].applymap(lambda x: isinstance(x, str)).any(axis=0)

        # Si hay valores problemáticos, mostrar un mensaje y no cargar el archivo
        if (negative_values.any() or str_values.any()):
            print(f"Valores de temperatura pronosticada para {fecha} son negativos o del tipo str. No se carga el archivo.")
            missing_data.append(fecha.strftime("%Y%m%d"))
        else:
            # Almacenar datos observados, pronosticados y la fecha en listas correspondientes
            datos_observados[fecha] = response_obs.text
            datos_pronostico.append(df)
            file_date.append(fecha.strftime('%Y%m%d'))
    else:
        # Si falta algún archivo, mostrar un mensaje y agregar la fecha a la lista de datos faltantes
        print(f"Hay archivos faltantes para la fecha {fecha}")
        missing_data.append(fecha.strftime("%Y%m%d"))
    
    return datos_pronostico, datos_observados, file_date

### Gestión de Datos de Temperatura: Descarga, Procesamiento y Manejo de Fechas

Se inicia con la definición de un diccionario `datos_observados` para almacenar el contenido de los archivos de temperatura observada, una lista `datos_pronostico` para almacenar datos de pronóstico, otra lista `file_date` para conservar las fechas de los archivos descargados, y una lista `missing_data` para registrar las fechas de archivos faltantes.

Se especifica un conjunto de nombres de columnas `COLUMN_NAMES` para el DataFrame que se creará a partir de los datos de pronóstico. La variable `fecha_anterior` se inicializa con la fecha de inicio, y a continuación, se inicia un bucle que itera sobre el rango de fechas entre `fecha_inicio` y `fecha_fin`.

Dentro del bucle, se calcula la fecha del archivo a descargar sumando un día a la fecha anterior. Se construyen las URL completas para los archivos de temperatura observada y pronosticada utilizando los componentes de la fecha. Luego, se realizan solicitudes HTTP para verificar la disponibilidad de ambos archivos.

Si algún archivo falta, se muestra un mensaje indicando la ausencia y se agrega la fecha a `missing_data`. También, en este caso, se verifica si es necesario cargar datos de temperatura observada de cuatro días atrás.

En caso de que ambos archivos estén disponibles, se decodifica y lee el archivo CSV de temperatura pronosticada. Se verifica la presencia de valores negativos o cadenas en los datos y, si se detectan, se emite un mensaje y se añade la fecha a la lista `missing_data`. Además, si hay menos de dos archivos faltantes, se intenta cargar datos de temperatura observada de cuatro días atrás utilizando la función `cargar_datos`.

Finalmente, la fecha anterior se actualiza para la próxima iteración del bucle.

In [388]:
datos_observados = {} # Diccionario para almacenar el contenido de los archivos de temperatura observada

datos_pronostico = [] # Lista para almacenar los datos de pronóstico

file_date = [] # Lista para almacenar las fechas de los archivos descargados

missing_data = []# Lista para almacenar las fechas de archivos faltantes

COLUMN_NAMES = ["Estacion", "Tmax", "Tmin"] # Nombres de columnas para el DataFrame

fecha_anterior = fecha_inicio # Inicializar la fecha anterior con la fecha de inicio

# Iterar sobre el rango de fechas
while fecha_anterior < fecha_fin:

    fecha = fecha_anterior + timedelta(days=1) # Calcular la fecha del archivo a descargar sumando un día a la fecha anterior

    # Desglosar la fecha en sus componentes: año, mes, día y nombre del mes en español
    año, mes, dia, mes_text = fecha.year, fecha.month, fecha.day, meses_espanol[fecha.month]

    # Crear el URL completo del archivo de temperatura observada utilizando los componentes de la fecha
    url_observado = f"{enlace_base_obs}/{año}/{mes_text}/temperaturas_{dia}_{mes}_{año}.txt"

    # Formar el enlace completo para el archivo CSV con la fecha actual y siguiente
    url_modelo = f"{enlace_base_mod}{fecha_anterior.strftime('%Y%m%d')}/{fecha.strftime('%Y%m%d')}.temperatura.csv"

    # Verificar si el archivo de temperatura observada y del modelo existe antes de descargarlo
    response_obs = requests.get(url_observado)
    response_mod = requests.get(url_modelo)

    # Verificar si ambos archivos están disponibles
    if response_obs.status_code == 200 and response_mod.status_code == 200:
        # Decodificar y leer los datos del archivo CSV de temperatura pronosticada
        mod_data = response_mod.content.decode('utf-8')
        df = pd.read_csv(io.StringIO(mod_data), header=None, names=COLUMN_NAMES)

        # Verificar la presencia de valores negativos o cadenas en el archivo de temperatura pronosticada
        negative_values = (df[COLUMN_NAMES[1:]] < 0).any(axis=0)
        str_values = df[COLUMN_NAMES[1:]].applymap(lambda x: isinstance(x, str)).any(axis=0)

        # Si hay valores problemáticos, mostrar un mensaje y no cargar el archivo
        if (negative_values.any() or str_values.any()):
            print(f"Valores de temperatura pronosticada para la fecha {fecha} son negativos o del tipo str. No se carga el archivo.")
            missing_data.append(fecha.strftime("%Y%m%d")) # Agregar la fecha del archivo faltante

            # Verificar si es necesario cargar datos de temperatura observada de cuatro días atrás
            if len(missing_data) < 2:
                fecha_sust = fecha_inicio  # Obtener la fecha siguiente
                fecha_sust_anterior = fecha_inicio - timedelta(days=1)
                datos_pronostico, datos_observados, file_date = cargar_datos(fecha_sust, fecha_sust_anterior, COLUMN_NAMES, enlace_base_obs, enlace_base_mod, meses_espanol)
            else:
                print(f"Hay más de dos archivos faltantes")
        else:
            # Almacenar datos observados, pronosticados y la fecha en listas correspondientes
            datos_observados[fecha] = response_obs.text
            datos_pronostico.append(df)
            file_date.append(fecha.strftime('%Y%m%d'))
    else:
        # Si falta algún archivo, mostrar un mensaje y agregar la fecha a la lista de datos faltantes
        print(f"Hay archivos faltantes para la fecha {fecha}")
        missing_data.append(fecha.strftime("%Y%m%d"))

        # Verificar si es necesario cargar datos de temperatura observada de cuatro días atrás
        if len(missing_data) < 2:
            fecha_sust = fecha_inicio  # Obtener la fecha siguiente
            fecha_sust_anterior = fecha_inicio - timedelta(days=1)
            datos_pronostico, datos_observados, file_date = cargar_datos(fecha_sust, fecha_sust_anterior, COLUMN_NAMES, enlace_base_obs, enlace_base_mod, meses_espanol)
        else:
            print(f"Hay más de dos archivos faltantes")

    # Actualizar la fecha anterior para la próxima iteración del ciclo
    fecha_anterior = fecha

## Dar formato a datos de Temperatura del WRF 1.5

Se crea una lista llamada `data_modelo` destinada a almacenar DataFrames del modelo. La iteración se realiza sobre las fechas y los DataFrames del pronóstico, emparejados mediante la función `zip`.

Dentro del bucle, se lleva a cabo una operación de preprocesamiento de datos, donde las columnas 'Tmax' y 'Tmin' se convierten a tipo numérico, gestionando posibles errores.

Posteriormente, se guardan los DataFrames resultantes en archivos CSV y TXT. Estos archivos se almacenan en la carpeta `WRF_Crudo` dentro de la ruta especificada por la variable `carpeta_modelo`, utilizando los nombres generados a partir de las fechas.

Finalmente, cada DataFrame procesado se agrega a la lista `data_modelo`.

In [389]:
# Lista para almacenar los DataFrames del modelo
data_modelo = []

# Iterar sobre las fechas y los DataFrames del pronóstico
for date, df in zip(file_date, datos_pronostico):
    
    # Convertir las columnas 'Tmax' y 'Tmin' a tipo numérico, manejando errores
    df[["Tmax", "Tmin"]] = df[["Tmax", "Tmin"]].apply(pd.to_numeric, errors='coerce')
    
    # Guardar el DataFrame en archivos CSV y TXT con el nombre generado
    df.to_csv(os.path.join(carpeta_modelo, 'WRF_Crudo', f"{date}.temperatura.csv"), index=False)
    df.to_csv(os.path.join(carpeta_modelo, 'WRF_Crudo', f"{date}.temperatura.txt"), index=False)
    
    # Agregar el DataFrame a la lista de DataFrames del modelo
    data_modelo.append(df)

## Procesamiento y Almacenamiento de Datos Observados
Se comienza estableciendo una lista vacía denominada `data_observada`, destinada a almacenar los DataFrames resultantes de los datos observados. Posteriormente, se implementa un bucle que itera a través de las parejas de `file_date` (fechas en formato de texto) y el contenido de archivos previamente almacenados en `datos_observados`.

Dentro de este bucle, el contenido de cada archivo se divide en líneas utilizando `"\n"` como separador. Se inicia una lista llamada `estaciones` para almacenar la información de las estaciones meteorológicas que será extraída del archivo. Además, se establece una variable llamada `seccion_encontrada` para rastrear si se ha encontrado la sección relevante en el archivo.

A continuación, el código procede a procesar cada línea del archivo. Si una línea contiene `"*****"` o `"....."`, indica que se ha encontrado la sección deseada. Si `seccion_encontrada` es `True` y la línea contiene un guion `"-"`, esto indica que contiene datos relevantes para una estación. Se dividen los datos utilizando espacios como separadores y se realiza una limpieza de datos, eliminando espacios en blanco y elementos vacíos. Si los datos extraídos no están vacíos y no comienzan con `"-----"`, se extraen la estación, la temperatura máxima y mínima.

A continuación, se verifica si los valores de tmax y tmin no son "NA" y, en caso de ser válidos, se convierten en tipo `float64`. El valor de la estación se convierte en tipo entero. Los datos de estación válidos se agregan a la lista `estaciones` como tuplas (estacion, tmax, tmin).

Una vez procesadas todas las líneas del archivo, la lista `estaciones` se convierte en un DataFrame de Pandas llamado `df_estaciones` con columnas "Estacion", "Tmax" y "Tmin". Posteriormente, se genera una ruta de carpeta `carpeta` donde se guardarán los archivos resultantes.

Finalmente, el DataFrame `df_estaciones` se agrega a la lista `data_observada`.

In [390]:
# Crear una lista para almacenar los DataFrames de datos de temperatura observada luego de ajustar el formato
data_observada = []

# Iterar sobre las fechas y DataFrames de temperatura observada
for date, archivo in zip(file_date, datos_observados.values()):
    # Dividir el contenido del archivo en líneas usando "\n" como separador
    lineas = archivo.strip().split("\n")[1:]  # Omitir la primera línea que es la cabecera

    # Inicializar una lista para almacenar los datos de las estaciones de este archivo
    estaciones = []

    # Variable para determinar si hemos encontrado la sección con estaciones
    seccion_encontrada = False

    # Procesar cada línea para extraer los datos de las estaciones
    for linea in lineas:
        if "*****" in linea:
            seccion_encontrada = True
        elif "....." in linea:
            seccion_encontrada = True
        elif seccion_encontrada and "-" in linea:
            datos_estacion = linea.split("   ")

            # Eliminar líneas vacías antes de procesar los datos
            datos_estacion = [dato.strip() for dato in datos_estacion if dato.strip()]

            # Asegurarnos de que la línea no esté vacía después de la eliminación
            if datos_estacion and not datos_estacion[0].startswith("-----"):
                estacion = datos_estacion[0].split("-")[0]
                tmax = datos_estacion[4]  # Corregir el índice para obtener la longitud
                tmin = datos_estacion[5]   # Corregir el índice para obtener la latitud
                
                # Solo usar las lineas donde tmax, tmin es diferente a 'NA'
                if tmax != "NA" and tmin != "NA":
                    tmax = float(tmax)# Convertir el valor a tipo float64
                    tmin = float(tmin)# Convertir el valor a tipo float64
                    estacion = int(estacion)# Convertir el valor a tipo int
                    
                    # Agregar los datos de la estación, tmax y tmin a la lista estaciones
                    estaciones.append((estacion, tmax, tmin))

    # Convertir la lista de estaciones en un DataFrame de Pandas para este archivo
    df_estaciones = pd.DataFrame(estaciones, columns=["Estacion", "Tmax", "Tmin"])

    # Guardar el DataFrame en un archivo CSV con el nombre generado
    df_estaciones.to_csv(os.path.join(carpeta_observado, 'Crudo', f"{date}.temperatura.csv"), index=False)
    df_estaciones.to_csv(os.path.join(carpeta_observado, 'Crudo', f"{date}.temperatura.txt"), index=False)

    # Agregar el DataFrame con el nuevo formato a la lista de DataFrames
    data_observada.append(df_estaciones)

## Crear Función para Cargar las Tablas de la Climatología Mensual

La función `cargar_clim` está diseñada para facilitar la carga de tablas de climatología mensual de la temperatura desde un archivo Excel. El usuario debe proporcionar la ruta del archivo Excel y una lista de nombres de hojas que desea cargar. La función utiliza la biblioteca Pandas para leer cada hoja especificada, convierte los nombres de las columnas a cadenas de texto y organiza los resultados en un diccionario. Este diccionario tiene como claves los nombres de las hojas y como valores los DataFrames correspondientes, permitiendo un acceso sencillo y organizado a los datos climatológicos cargados. 

In [391]:
def cargar_clim(ruta_excel, hojas_excel):
    """
    Cargar las tablas de la climatología mensual de la temperatura observada
    
    Parameters:
        ruta_excel (str): Ruta del archivo Excel que contiene las tablas.
        hojas_excel (list): Lista de nombres de hojas de Excel a cargar.

    Returns:
        dict: Un diccionario que contiene las tablas cargadas. Las claves son los nombres de las hojas y los valores son DataFrames de Pandas.
    """
    # Crear un diccionario para almacenar las tablas de la climatología mensual (temperatura observada) cargadas desde Excel
    clim = {}
    
    # Iterar sobre la lista de nombres de hojas de Excel a cargar
    for hoja in hojas_excel:
        # Cargar la tabla desde la hoja de Excel especificada
        tabla = pd.read_excel(ruta_excel, sheet_name=hoja)
        
        # Convertir los nombres de las columnas a cadenas de texto
        tabla.columns = tabla.columns.astype(str)
        
        # Agregar la tabla al diccionario utilizando el nombre de la hoja como clave
        clim[hoja] = tabla

    return clim

### Carga de Tablas Climatología Mensual Observada

En el código proporcionado, se define la variable `ruta_observado` como la ruta completa de un archivo Excel que contiene la climatología mensual de la temperatura observada. Además, se especifica una lista de nombres de hojas relevantes dentro de ese archivo. Luego, se utiliza la función `cargar_clim` con los parámetros `ruta_observado` y `hojas_observado` para cargar las tablas correspondientes desde el archivo Excel.

In [392]:
# Ruta del archivo Excel que contiene la climatología mensual de temperatura observada.
ruta_observado = 'C:/Users/arias/OneDrive/Documentos/UCR/TFG/Climatologia/Observado/Temperatura/Climatologia_Mensual_Observada_Temperatura.xlsx'
hojas_observado = ['tmax_mean', 'tmin_mean', 'tmax_max', 'tmin_max', 'tmax_min', 'tmin_min']

#Cargar las tablas de la climatología mensual de la temperatura observada
clim_observado = cargar_clim(ruta_observado, hojas_observado)

### Carga de Tablas Climatología Mensual del Modelo

En el código proporcionado, se define la variable `ruta_modelo` como la ruta completa de un archivo Excel que contiene la climatología mensual de la temperatura pronosticada. Además, se especifica una lista de nombres de hojas relevantes dentro de ese archivo. Luego, se utiliza la función `cargar_clim` con los parámetros `ruta_modelo` y `hojas_modelo` para cargar las tablas correspondientes desde el archivo Excel.

In [393]:
# Ruta del archivo Excel que contiene la climatología mensual de temperatura observada.
ruta_modelo = 'C:/Users/arias/OneDrive/Documentos/UCR/TFG/Climatologia/Modelo/Temperatura/Climatologia_Mensual_Modelo_Temperatura.xlsx'
hojas_modelo = ['tmax_mean', 'tmin_mean']

#Cargar las tablas de la climatología mensual de la temperatura observada
clim_modelo = cargar_clim(ruta_modelo, hojas_modelo)

## Control de Calidad de Datos con Climatología Mensual Observada

### Datos de Pronóstico

El código comienza verificando si la variable `data_modelo` no es `None`, indicando la presencia de datos. Si `data_modelo` es `None`, se imprime un mensaje indicando que no se pudieron obtener los datos adecuadamente. Luego, se emplea un bucle `for` para iterar sobre las fechas (`date`) en `file_date` y los datos (`file`) en `data_modelo`.

En cada iteración, se convierte la fecha en formato de cadena (`date`) a un objeto `datetime`. Además, se construyen las rutas completas para dos archivos CSV (`tabla_cc_tmax` y `tabla_cc_tmin`) que contendrán tablas de control de calidad. Estas rutas incluyen la carpeta especificada y el año de la fecha actual. Se verifica la existencia de los archivos `ruta_tmax_cc` y `ruta_tmin_cc` en el sistema de archivos. Si existen, se cargan los datos en dos DataFrames; de lo contrario, se crean los DataFrames.

Luego, se verifica si la fecha actual ya existe en las tablas `tabla_cc_tmax` o `tabla_cc_tmin`. En caso afirmativo, se eliminan las filas correspondientes. Se crean nuevas filas en los DataFrames `tabla_cc_tmax` y `tabla_cc_tmin` con valores iniciales vacíos y la fecha actual.

A continuación, se inicia un bucle para iterar sobre las estaciones presentes en `tabla_cc_tmax` (`tabla_cc_tmin`). Para cada estación y mes correspondiente, se obtienen los límites climatológicos mensuales de temperatura máxima (`tmax_min` y `tmax_max`) desde el DataFrame `clim_observado`. Se realiza una operación similar para obtener los límites climatológicos de temperatura mínima (`tmin_min` y `tmin_max`).

Se filtran las filas del DataFrame `file` que corresponden a la estación actual. Se verifica si hay datos disponibles para la estación en el DataFrame `file`. Si hay datos, se obtienen los valores pronosticados de temperatura máxima (`Tmax`) y mínima (`Tmin`) para la estación actual desde el DataFrame `file`.

Las temperaturas pronosticadas (`Tmax` y `Tmin`) se ajustan según los límites climatológicos. Si el valor pronosticado excede el límite máximo, se ajusta al valor máximo; si es menor que el límite mínimo, se ajusta al valor mínimo. Si está dentro de los límites, se asigna un guion (`-`). Si no hay datos para la estación en el DataFrame `file`, se asigna el valor "x" a las columnas correspondientes en `tabla_tmax_fila` y `tabla_tmin_fila`. 
Posteriormente, los diccionarios `tabla_tmax_fila` y `tabla_tmin_fila` se convierten en DataFrames.

Finalmente, las tablas de control de calidad actualizadas se guardan en archivos CSV y TXT correspondientes a las temperaturas máximas y mínimas.

### Datos de Pronostico

In [394]:
# Verificar si data_modelo es None
if data_modelo is not None:
    for date, file in zip(file_date, data_modelo):

        fecha = datetime.strptime(date, "%Y%m%d")
        
        # Ruta completa del archivo que contendrá la tabla de control de calidad
        ruta_tmax_cc = os.path.join(carpeta, f"tabla_cc_tmax_{fecha.year}.csv")
        ruta_tmin_cc = os.path.join(carpeta, f"tabla_cc_tmin_{fecha.year}.csv")

        # Si el archivo no existe, se crea la tabla vacía y se descargan y procesan los datos
        if os.path.exists(ruta_tmax_cc) and os.path.exists(ruta_tmin_cc):
            # Continuar con el procesamiento
            tabla_cc_tmax = pd.read_csv(ruta_tmax_cc)
            tabla_cc_tmin = pd.read_csv(ruta_tmin_cc)
        else:
            # Si el archivo existe, cargar la tabla y procesar el día siguiente
            tabla_cc_tmax = pd.DataFrame(columns=['FECHA'] + [str(col) for col in data_modelo[0]['Estacion']])
            tabla_cc_tmin = pd.DataFrame(columns=['FECHA'] + [str(col) for col in data_modelo[0]['Estacion']])

        # Convertir fecha_siguiente_str a np.int64 para hacer la comparación correctamente
        fecha_siguiente_int = np.int64(date)

        # Verificar si la fecha ya existe en la tabla
        if (tabla_cc_tmax["FECHA"] == fecha_siguiente_int).any() or (tabla_cc_tmin["FECHA"] == fecha_siguiente_int).any():
            # Si la fecha ya existe, obtener el índice de la fila existente
            idx_fecha_existente_tmax = tabla_cc_tmax.loc[tabla_cc_tmax["FECHA"] == fecha_siguiente_int].index[0]
            idx_fecha_existente_tmin = tabla_cc_tmin.loc[tabla_cc_tmin["FECHA"] == fecha_siguiente_int].index[0]

            # Eliminar la fila existente a partir del índice
            if idx_fecha_existente_tmax is not None:
                tabla_cc_tmax = tabla_cc_tmax.drop(idx_fecha_existente_tmax)

            if idx_fecha_existente_tmin is not None:
                tabla_cc_tmin = tabla_cc_tmin.drop(idx_fecha_existente_tmin)

        # Crear una nueva fila con la fecha actual
        tabla_tmax_fila = {col: "" for col in tabla_cc_tmax.columns}
        tabla_tmax_fila["FECHA"] = date
        
        tabla_tmin_fila = {col: "" for col in tabla_cc_tmin.columns}
        tabla_tmin_fila["FECHA"] = date

        columnas_temperatura = ['tmax_min', 'tmax_max', 'tmin_min', 'tmin_max']

        for estacion in tabla_cc_tmax.columns[1:]:
            if any(estacion in clim_observado[col].columns for col in columnas_temperatura):

                # Obtener el valor de climatología mensual de temperatura observada (Tmax) para la estación y el mes correspondiente
                valor_tmax_min = clim_observado['tmax_min'].loc[clim_observado['tmax_min']["mes"] == fecha.month, estacion].values[0]
                valor_tmax_max = clim_observado['tmax_max'].loc[clim_observado['tmax_max']["mes"] == fecha.month, estacion].values[0]

                # Obtener el valor de climatología mensual de temperatura observada (Tmin) para la estación y el mes correspondiente
                valor_tmin_min = clim_observado['tmin_min'].loc[clim_observado['tmin_min']["mes"] == fecha.month, estacion].values[0]
                valor_tmin_max = clim_observado['tmin_max'].loc[clim_observado['tmin_max']["mes"] == fecha.month, estacion].values[0]

                # Verificar si existen filas que coincidan con la estación en file
                filas_estacion = file.loc[file["Estacion"].astype(str).str.strip() == estacion.strip()]

                # Verificar que la fila para la 'estación' en temperatura pronosticada no esté vacía
                if not filas_estacion.empty:
                    # Obtener el valor de temperatura pronosticada (Tmax, Tmin) para la estación y el mes correspondiente
                    valor_tmax_modelo = filas_estacion["Tmax"].iloc[0]
                    valor_tmin_modelo = filas_estacion["Tmin"].iloc[0]

                    # Ajustar la temperatura pronosticada (tmax) con los limites de la climatología mensual de temperatura observada (tmax)
                    if valor_tmax_modelo >= valor_tmax_max:
                        tabla_tmax_fila[estacion] = f"{valor_tmax_max}({valor_tmax_modelo}>Max)"
                        file.loc[filas_estacion.index, "Tmax"] = valor_tmax_max
                    elif valor_tmax_modelo < valor_tmax_min:
                        tabla_tmax_fila[estacion] = f"{valor_tmax_min}({valor_tmax_modelo}<Min)"
                        file.loc[filas_estacion.index, "Tmax"] = valor_tmax_min
                    else:
                        tabla_tmax_fila[estacion] = "-" # Si el valor está dentro de los limites, asignar "-"

                    # Ajustar la temperatura pronosticada (tmin) con los limites de la climatología mensual de temperatura observada (tmin)
                    if valor_tmin_modelo >= valor_tmin_max:
                        tabla_tmin_fila[estacion] = f"{valor_tmin_max}({valor_tmin_modelo}>Max)"
                        file.loc[filas_estacion.index, "Tmin"] = valor_tmin_max
                    elif valor_tmin_modelo < valor_tmin_min:
                        tabla_tmin_fila[estacion] = f"{valor_tmin_min}({valor_tmin_modelo}<Min)"
                        file.loc[filas_estacion.index, "Tmin"] = valor_tmin_min
                    else:
                        tabla_tmin_fila[estacion] = "-" # Si el valor está dentro de los limites, asignar "-"
                else:
                    # Si no hay valor para la estación en data_modelo, asignar "x"
                    tabla_tmax_fila[estacion] = "x"
                    tabla_tmin_fila[estacion] = "x"
                    print(f'La estación {estacion} no existe en los datos de pronostico de temperatura del WRF1.5')
            else:
                nueva_fila[estacion] = "x"
                print(f'La estación {estacion} no existe en las columnas de la Climatología de Temperatura Mensual Observada')

        # Convertir los diccionarios en Pandas DataFrames
        tabla_tmax_fila = pd.DataFrame.from_dict(tabla_tmax_fila, orient='index').T
        tabla_tmin_fila = pd.DataFrame.from_dict(tabla_tmin_fila, orient='index').T

        # Concatenar los DataFrames
        tabla_cc_tmax = pd.concat([tabla_cc_tmax, tabla_tmax_fila], ignore_index=True)
        tabla_cc_tmin = pd.concat([tabla_cc_tmin, tabla_tmin_fila], ignore_index=True)

        # Guardar la clim_observado de cc y data_modelo en un archivo CSV y TXT
        tabla_cc_tmax.to_csv(f"{carpeta}/tabla_cc_tmax_{fecha.year}.csv", index=False)
        tabla_cc_tmax.to_csv(os.path.join(carpeta, f"tabla_cc_tmax_{fecha.year}.txt"), index=False)

        tabla_cc_tmin.to_csv(f"{carpeta}/tabla_cc_tmin_{fecha.year}.csv", index=False)
        tabla_cc_tmin.to_csv(os.path.join(carpeta, f"tabla_cc_tmin_{fecha.year}.txt"), index=False)
else:
        print("No se pudieron obtener los datos correctamente ya que el archivo que contiene los datos de pronostico (data_modelo) está vacio.")

### Datos Observados
Primero se comprueba si la variable `data_observada` no es `None`. Si tiene datos, procede con el procesamiento. En caso contrario, imprime un mensaje indicando que no se pudieron obtener los datos correctamente.

Para cada fecha en `file_date` y su respectivo archivo en `data_observada_cc`: Convierte la fecha a un objeto `datetime`, genera la ruta completa del archivo que contendrá la tabla de control de calidad (`ruta_archivo_tmax_cc` y `ruta_archivo_tmin_cc`), verifica si el archivo ya existe. Si no existe, crea una tabla vacía. Si existe, carga la tabla existente. Convierte la fecha a un número entero para facilitar la comparación con fechas en la tabla.

Se verifica si la fecha ya existe en la tabla de Control de Calidad. Si existe, elimina la fila existente. Srea una nueva fila con la fecha y procesa los datos de temperatura pronosticada y observada para cada estación.

Par}a el control de calidad de datos de temperatura observada, se compara los valores de temperatura observada con los límites de la climatología mensual de temperatura observada y se actualiza los valores en el archivo original (`file`), marcando aquellos que no cumplen con los límites como `NaN`.

Guarda la tabla ajustada en un archivo CSV y TXT. Guarda el archivo original con los datos actualizados en una carpeta específica, utilizando el nombre de la fecha como parte del nombre del archivo.

Imprime mensajes de advertencia si una estación no existe en los datos observados de temperatura o en las columnas de la climatología mensual observada. Realiza el mismo procedimiento para Temperatura máxima y temperatura mínima.

In [395]:
# Verificar si data_modelo es None
if data_observada is not None:
    data_observada_cc = copy.deepcopy(data_observada)
    for date, file in zip(file_date, data_observada_cc):

        fecha = datetime.strptime(date, "%Y%m%d")
        
        # Ruta completa del archivo que contendrá la tabla de control de calidad
        ruta_tmax_cc = os.path.join(carpeta, f"tabla_cc_tmax_observado_{fecha.year}.csv")
        ruta_tmin_cc = os.path.join(carpeta, f"tabla_cc_tmin_observado_{fecha.year}.csv")

        # Si el archivo no existe, se crea la tabla vacía y se descargan y procesan los datos
        if os.path.exists(ruta_tmax_cc) and os.path.exists(ruta_tmin_cc):
            # Continuar con el procesamiento
            tabla_cc_tmax_observado = pd.read_csv(ruta_tmax_cc)
            tabla_cc_tmin_observado = pd.read_csv(ruta_tmin_cc)
        else:
            # Si el archivo existe, cargar la tabla y procesar el día siguiente
            tabla_cc_tmax_observado = pd.DataFrame(columns=['FECHA'] + [str(col) for col in data_observada_cc[0]['Estacion']])
            tabla_cc_tmin_observado = pd.DataFrame(columns=['FECHA'] + [str(col) for col in data_observada_cc[0]['Estacion']])

        # Convertir fecha_siguiente_str a np.int64 para hacer la comparación correctamente
        fecha_siguiente_int = np.int64(date)

        # Verificar si la fecha ya existe en la tabla
        if (tabla_cc_tmax_observado["FECHA"] == fecha_siguiente_int).any() or (tabla_cc_tmin_observado["FECHA"] == fecha_siguiente_int).any():
            # Si la fecha ya existe, obtener el índice de la fila existente
            idx_fecha_existente_tmax = tabla_cc_tmax_observado.loc[tabla_cc_tmax_observado["FECHA"] == fecha_siguiente_int].index[0]
            idx_fecha_existente_tmin = tabla_cc_tmin_observado.loc[tabla_cc_tmin_observado["FECHA"] == fecha_siguiente_int].index[0]

            # Eliminar la fila existente a partir del índice
            if idx_fecha_existente_tmax is not None:
                tabla_cc_tmax_observado = tabla_cc_tmax_observado.drop(idx_fecha_existente_tmax)

            if idx_fecha_existente_tmin is not None:
                tabla_cc_tmin_observado = tabla_cc_tmin_observado.drop(idx_fecha_existente_tmin)

        # Crear una nueva fila con la fecha actual
        tabla_tmax_fila = {col: "" for col in tabla_cc_tmax_observado.columns}
        tabla_tmax_fila["FECHA"] = date
        
        tabla_tmin_fila = {col: "" for col in tabla_cc_tmin_observado.columns}
        tabla_tmin_fila["FECHA"] = date

        olumnas_temperatura = ['tmax_min', 'tmax_max', 'tmin_min', 'tmin_max']

        for estacion in tabla_cc_tmax_observado.columns[1:]:
            if any(estacion in clim_observado[col].columns for col in columnas_temperatura):
                # Obtener el valor de climatología mensual de temperatura observada (Tmax) para la estación y el mes correspondiente
                valor_tmax_min = clim_observado['tmax_min'].loc[clim_observado['tmax_min']["mes"] == fecha.month, estacion].values[0]
                valor_tmax_max = clim_observado['tmax_max'].loc[clim_observado['tmax_max']["mes"] == fecha.month, estacion].values[0]

                # Obtener el valor de climatología mensual de temperatura observada (Tmin) para la estación y el mes correspondiente
                valor_tmin_min = clim_observado['tmin_min'].loc[clim_observado['tmin_min']["mes"] == fecha.month, estacion].values[0]
                valor_tmin_max = clim_observado['tmin_max'].loc[clim_observado['tmin_max']["mes"] == fecha.month, estacion].values[0]

                # Verificar si existen filas que coincidan con la estación en file
                filas_estacion = file.loc[file["Estacion"].astype(str).str.strip() == estacion.strip()]

                # Verificar que la fila para la 'estación' en temperatura pronosticada no esté vacía
                if not filas_estacion.empty:
                    # Obtener el valor de temperatura pronosticada (Tmax, Tmin) para la estación y el mes correspondiente
                    valor_tmax_modelo = filas_estacion["Tmax"].iloc[0]
                    valor_tmin_modelo = filas_estacion["Tmin"].iloc[0]

                    # Ajustar la temperatura pronosticada (tmax) con los limites de la climatología mensual de temperatura observada (tmax)
                    if valor_tmax_modelo >= valor_tmax_max:
                        tabla_tmax_fila[estacion] = f"{valor_tmax_max}({valor_tmax_modelo}>Max)"
                        file.loc[filas_estacion.index, "Tmax"] = np.nan
                    elif valor_tmax_modelo < valor_tmax_min:
                        tabla_tmax_fila[estacion] = f"{valor_tmax_min}({valor_tmax_modelo}<Min)"
                        file.loc[filas_estacion.index, "Tmax"] = np.nan
                    else:
                        tabla_tmax_fila[estacion] = "-" # Si el valor está dentro de los limites, asignar "-"

                    # Ajustar la temperatura pronosticada (tmin) con los limites de la climatología mensual de temperatura observada (tmin)
                    if valor_tmin_modelo >= valor_tmin_max:
                        tabla_tmin_fila[estacion] = f"{valor_tmin_max}({valor_tmin_modelo}>Max)"
                        file.loc[filas_estacion.index, "Tmin"] = np.nan
                    elif valor_tmin_modelo < valor_tmin_min:
                        tabla_tmin_fila[estacion] = f"{valor_tmin_min}({valor_tmin_modelo}<Min)"
                        file.loc[filas_estacion.index, "Tmin"] = np.nan
                    else:
                        tabla_tmin_fila[estacion] = "-" # Si el valor está dentro de los limites, asignar "-"
                else:
                    # Si no hay valor para la estación en data_modelo, asignar "x"
                    tabla_tmax_fila[estacion] = "x"
                    tabla_tmin_fila[estacion] = "x"
                    print(f'La estación {estacion} no existe en los datos de observados de temperatura en la fecha {date}')
            else:
                tabla_tmax_fila[estacion] = "x"
                tabla_tmin_fila[estacion] = "x"
                print(f'La estación {estacion} no existe en las columnas de la Climatología de Temperatura Mensual Observada')

        # Convertir los diccionarios en Pandas DataFrames
        tabla_tmax_fila = pd.DataFrame.from_dict(tabla_tmax_fila, orient='index').T
        tabla_tmin_fila = pd.DataFrame.from_dict(tabla_tmin_fila, orient='index').T

        # Concatenar los DataFrames
        tabla_cc_tmax_observado = pd.concat([tabla_cc_tmax_observado, tabla_tmax_fila], ignore_index=True)
        tabla_cc_tmin_observado = pd.concat([tabla_cc_tmin_observado, tabla_tmin_fila], ignore_index=True)

        # Guardar la clim_observado de cc y data_modelo en un archivo CSV y TXT
        tabla_cc_tmax_observado.to_csv(f"{carpeta}/tabla_cc_tmax_observado_{fecha.year}.csv", index=False)
        tabla_cc_tmax_observado.to_csv(os.path.join(carpeta, f"tabla_cc_tmax_observado_{fecha.year}.txt"), index=False)

        tabla_cc_tmin_observado.to_csv(f"{carpeta}/tabla_cc_tmin_observado_{fecha.year}.csv", index=False)
        tabla_cc_tmin_observado.to_csv(os.path.join(carpeta, f"tabla_cc_tmin_observado_{fecha.year}.txt"), index=False)
        
        # Guardar el DataFrame en un archivo CSV con el nombre generado
        file.to_csv(os.path.join(carpeta_observado, 'Editado', f"{date}.temperatura.csv"), index=False)
        file.to_csv(os.path.join(carpeta_observado, 'Editado', f"{date}.temperatura.txt"), index=False)
else:
        print("No se pudieron obtener los datos correctamente ya que el archivo que contiene los datos de pronostico (data_modelo) está vacio.")

La estación 69695 no existe en las columnas de la Climatología de Temperatura Mensual Observada
La estación 84221 no existe en las columnas de la Climatología de Temperatura Mensual Observada
La estación 69753 no existe en las columnas de la Climatología de Temperatura Mensual Observada
La estación 74059 no existe en las columnas de la Climatología de Temperatura Mensual Observada
La estación 72193 no existe en las columnas de la Climatología de Temperatura Mensual Observada
La estación 76057 no existe en las columnas de la Climatología de Temperatura Mensual Observada
La estación 90013 no existe en las columnas de la Climatología de Temperatura Mensual Observada
La estación 84255 no existe en las columnas de la Climatología de Temperatura Mensual Observada
La estación 73175 no existe en las columnas de la Climatología de Temperatura Mensual Observada
La estación 84293 no existe en las columnas de la Climatología de Temperatura Mensual Observada
La estación 69695 no existe en las colum

## Pasos previos al Ajuste de Datos con Climatología Mensual

### Crear un MAPEO que permite relacionar las cabeceras de cantón con estaciones especificas

* Se define un diccionario llamado `mapeo`. Cada clave del diccionario representa el nombre de un cantón de Costa Rica, y el valor asociado a esa clave es una lista de estaciones que se asocian por ubicación a ese cantón.

In [396]:
mapeo = {
    'ALAJUELA': ['84185', '84187'],
    'ABANGARES': ['78031'],
    'ACOSTA': ['88055'],
    'AGUIRRE': ['90015'],
    'ALAJUELITA': ['84203'],
    'ALFARO_RUIZ': ['69737'],
    'ALVARADO': ['73141'],
    'ASERRI': ['84215', '84217'],
    'ATENAS': ['84225'],
    'BAGACES': ['76055'],
    'BARVA': ['84243', '84197'],
    'BELEN': ['84199'],
    'BUENOS_AIRES': ['98087'],
    'CANAS': ['76055'],
    'CARRILLO': ['74067'],
    'CARTAGO': ['73123'],
    'CD_QUESADA': ['69743', '69701'],
    'CORREDORES': ['100651'],
    'COTO_BRUS': ['98107', '98109', '98075'],
    'CURRIDABAT': ['84203'],
    'DESAMPARADOS': ['84203'],
    'DOTA': ['88051'],
    'EL_GUARCO': ['73123', '73171'],
    'ESPARZA': ['80013'],
    'ESCAZU': ['84231', '84193'],
    'FLORES': ['84199', '84197'],
    'GARABITO': ['86013'],
    'GOICOECHEA': ['84139'],
    'GOLFITO': ['100643'],
    'GRECIA': ['84295'],
    'GUACIMO': ['73145'],
    'GUAPILES': ['73147'],
    'GUATUSO': ['69747', '69749'],
    'HEREDIA': ['84193', '84243', '84197'],
    'HOJANCHA': ['72185'],
    'JIMENEZ': ['73149'],
    'LA_CRUZ': ['72191'],
    'LA_UNION': ['84181', '73129', '84249'],
    'LEON_CORTEZ': ['88051', '88047'],
    'LIBERIA': ['74063'],
    'LIMON': ['81005'],
    'LOS_CHILES': ['69633', '69713'],
    'MATINA': ['73159', '83007'],
    'MONTES_DE_OCA': ['84139', '84203'],
    'MONTES_DE_ORO': ['78033'],
    'MORA': ['84209', '84283'],
    'MORAVIA': ['84139'],
    'NANDAYURE': ['72189'],
    'NARANJO': ['84239'],
    'NICOYA': ['72165', '72183'],
    'OREAMUNO': ['73123', '73129'],
    'OROTINA': ['82011', '82019', '82017'],
    'OSA': ['100655'],
    'PALMARES': ['84239'],
    'PARAISO': ['73123'],
    'PARRITA': ['88049'],
    'PAVAS': ['84193', '84285'],
    'PEREZ_ZELEDON': ['98097', '94015'],
    'POAS': ['84189', '84187'],
    'POCOCI': ['73147', '73169'],
    'PTO_VIEJO': ['85025', '85023'],
    'PUNTARENAS': ['78027'],
    'PURISCAL': ['84209'],
    'QUEPOS': ['90015'],
    'SAN_CARLOS': ['69743', '69701'],
    'SAN_ISIDRO': ['84243'],
    'SAN_JOSE': ['84141'],
    'SAN_MARCOS': ['88051'],
    'SAN_MATEO': ['82011'],
    'SAN_PABLO': ['84243', '84193'],
    'SAN_RAFAEL': ['84243'],
    'SAN_RAMON': ['84239'],
    'SANTA_ANA': ['84219'],
    'SANTA_BARBARA': ['84197'],
    'SANTA_CRUZ': ['74053'],
    'SANTO_DOMINGO': ['84193'],
    'SARAPIQUI': ['69725'],
    'SIQUIRRES': ['73159'],
    'SIXAOLA': ['87013'],
    'TALAMANCA': ['85021'],
    'TARRAZU': ['88051'],
    'TIBAS': ['84141'],
    'TILARAN': ['76063'],
    'TORTUGUERO': ['71023'],
    'TURRIALBA': ['73151', '73155', '73167'],
    'TURRUBARES': ['84225'],
    'UPALA': ['69679', '69647'],
    'VALVERDE_VEGA': ['84239'],
    'VASQUEZ': ['84207', '84213']
}

### Mapeo Invertido

Se construye un diccionario inverso llamado `mapeo_invertido`. Inicialmente, se tiene un diccionario `mapeo` que asocia cantones con estaciones meteorológicas. 

El código realiza un bucle sobre cada elemento del diccionario `mapeo`, en cada iteración para cada estación asociado a un canton, se verifica si dicho código ya está presente en el diccionario inverso `mapeo_invertido`. Si la estación ya existe, se agrega el cantón actual a la lista existente de cantones asociadas a esa estación. En caso contrario, se crea una nueva entrada en `mapeo_invertido` con la estación como clave y una lista que contiene el cantón actual como único elemento.

El resultado final es un diccionario inverso que asocia cada estación con una lista de cantones de Costa Rica.

In [397]:
mapeo_invertido = {}

for ubicacion, codigos in mapeo.items():
    for codigo in codigos:
        if codigo in mapeo_invertido:
            mapeo_invertido[codigo].append(ubicacion)
        else:
            mapeo_invertido[codigo] = [ubicacion]

### Lista de nombres de las cabeceras de cantón

* Crear una lista llamada `names_cantones` que contiene los nombres de cada cantón acomodados en el mismo orden que las columnas del diccionario `clim_modelo`. Lo anterior permitirá renombrar las columnas de `clim_modelo` luego para que tengan un formato correcto y homogeneo. 

In [398]:
names_cantones = ['ALAJUELA', 'ABANGARES', 'ACOSTA', 'AGUIRRE', 'ALAJUELITA','ALFARO_RUIZ', 'ALVARADO', 'ASERRI', 'ATENAS', 
           'BAGACES', 'BARVA','BELEN', 'BUENOS_AIRES', 'CANAS', 'CARRILLO', 'CARTAGO', 'CD_QUESADA','CORREDORES', 
           'COTO_BRUS', 'CURRIDABAT', 'DESAMPARADOS', 'DOTA','EL_GUARCO', 'ESPARZA', 'ESCAZU', 'FLORES', 'GARABITO', 
           'GOICOECHEA','GOLFITO', 'GRECIA', 'GUACIMO', 'GUAPILES', 'GUATUSO', 'HEREDIA','HOJANCHA', 'JIMENEZ', 'LA_CRUZ', 
           'LA_UNION', 'LEON_CORTEZ', 'LIBERIA','LIMON', 'LOS_CHILES', 'MATINA', 'MONTES_DE_OCA', 'MONTES_DE_ORO','MORA', 
           'MORAVIA', 'NANDAYURE', 'NARANJO', 'NICOYA', 'OREAMUNO','OROTINA', 'OSA', 'PALMARES', 'PARAISO', 'PARRITA', 
           'PAVAS','PEREZ_ZELEDON', 'POAS', 'POCOCI', 'PTO_VIEJO', 'PUNTARENAS','PURISCAL', 'QUEPOS', 'SAN_CARLOS', 
           'SAN_ISIDRO', 'SAN_JOSE','SAN_MARCOS', 'SAN_MATEO', 'SAN_PABLO', 'SAN_RAFAEL', 'SAN_RAMON','SANTA_ANA', 
           'SANTA_BARBARA', 'SANTA_CRUZ', 'SANTO_DOMINGO','SARAPIQUI', 'SIQUIRRES', 'SIXAOLA', 'TALAMANCA', 'TARRAZU', 
           'TIBAS','TILARAN', 'TORTUGUERO', 'TURRIALBA', 'TURRUBARES', 'UPALA', 'VALVERDE_VEGA', 'VASQUEZ']

###  Integración y Promedio de datos por Estación usando el Mapeo Invertido

Se crea la función `process_clim_modelo_tabla` se realiza para dar formato a los datos promedio de la climatología mensual del modelo. Primero, la función toma la columna deseada (`tabla`) del DataFrame `clim_modelo` y la almacena en un nuevo DataFrame llamado `df`.

Posteriormente, crea un nuevo DataFrame llamado `Clim_Modelo_Mean` que servirá para almacenar los resultados finales procesados. La función itera sobre las columnas del diccionario inverso `mapeo_invertido`, el cual asocia cada estación meteorológica con la lista de cantones correspondientes.

En cada iteración, verifica si la lista de cantones asociada a una estación tiene solo un elemento. Si es así, asigna directamente los valores de la columna correspondiente de `df` a esa estación en `Clim_Modelo_Mean`. Si la lista tiene más de un elemento, calcula la media de las columnas asociadas para esa estación, manejando los valores nulos.

Finalmente, retorna el DataFrame resultante. Este proceso se utiliza para procesar dos conjuntos de datos de restas climatológicas correspondientes a las temperaturas máximas y mínimas, creando los DataFrames `Clim_Modelo_Tmax_Mean` y `Clim_Modelo_Tmin_Mean`.

In [399]:
def process_clim_modelo_tabla(tabla, names_cantones, mapeo_invertido):
    # Convertir el DataFrame 'resta' a una nueva tabla llamada Tabla_Resta_Clim
    df = clim_modelo[tabla]
    df = pd.DataFrame(df)
    df = df.drop('mes', axis=1)
    df.columns = names_cantones

    # Crear un nuevo DataFrame para almacenar los resultados finales
    Clim_Modelo_Mean = pd.DataFrame()

    # Iterar sobre las columnas de mapeo_invertido
    for estacion, cantones in mapeo_invertido.items():
        # Verificar si hay un solo elemento en la lista
        if len(cantones) == 1:
            Clim_Modelo_Mean[estacion] = df[cantones[0]]  # Asignar directamente el valor correspondiente
        else:
            # Si hay más de un elemento, calcular la media y manejar valores nulos
            Clim_Modelo_Mean[estacion] = df[cantones].mean(axis=1, skipna=True)

    return Clim_Modelo_Mean

Clim_Modelo_Tmax_Mean = process_clim_modelo_tabla('tmax_mean', names_cantones, mapeo_invertido)
Clim_Modelo_Tmin_Mean = process_clim_modelo_tabla('tmin_mean', names_cantones, mapeo_invertido)

### Obtener la diferencia entre la climatología del modelo y la observada para cada cantón.

La función `process_clim_resta_tabla`  realiza una operación entre `Clim_Modelo_Mean` y `Clim_Observ_Mean`, y luego agrega la columna 'mes' del DataFrame `clim_observado` al DataFrame resultante llamado `Clim_Resta`. La resta se realiza elemento por elemento entre las dos tablas de datos. Luego, la funcion se llama para crear dos dataframes de temperatura máxima y mínima, creando los DataFrames `Clim_Resta_Tmax` y `Clim_Resta_Tmin`.

In [400]:
def process_clim_resta_tabla(Clim_Modelo_Mean, clim_observado, tabla):
    
    Clim_Observ_Mean = clim_observado[tabla]

    Clim_Resta = pd.DataFrame()

    for estacion in Clim_Modelo_Mean.columns:  # Corregir el nombre de la variable
        resta = Clim_Modelo_Mean[estacion] - Clim_Observ_Mean[estacion]
        Clim_Resta[estacion] = resta  # Usar directamente el DataFrame para asignar columnas

    # Agregar la columna 'mes'
    Clim_Resta['mes'] = clim_observado[tabla]['mes']
    
    return Clim_Resta

Clim_Resta_Tmax = process_clim_resta_tabla(Clim_Modelo_Tmax_Mean, clim_observado, 'tmax_mean')
Clim_Resta_Tmin = process_clim_resta_tabla(Clim_Modelo_Tmin_Mean, clim_observado, 'tmin_mean')

## Ajuste de Datos con Climatología Mensual Observada y Modelada

El fragmento de código realiza una serie de procesamientos y ajustes en los datos de temperatura pronosticada (`Tmax` y `Tmin`) contenidos en el archivo `data_modelo`. Primero, se verifica si `data_modelo` no es `None`. Si hay datos disponibles, se procede a un bucle que itera sobre las fechas (`date`) y los archivos (`file`) en `file_date` y `data_modelo`, respectivamente.

Para cada fecha, se convierte la cadena de fecha en un objeto `datetime`. Luego, se construyen las rutas completas para dos archivos CSV que contendrán tablas de ajuste para temperaturas máximas (`tabla_ajuste_tmax`) y mínimas (`tabla_ajuste_tmin`). Si los archivos ya existen, se cargan en DataFrames (`tabla_ajuste_tmax` y `tabla_ajuste_tmin`). En caso contrario, se crean DataFrames vacíos con columnas específicas.

Se verifica si la fecha actual ya existe en las tablas de ajuste. Si es así, se eliminan las filas existentes correspondientes. Se crea una nueva fila en las tablas de ajuste con la fecha actual y se itera sobre las estaciones meteorológicas presentes en las tablas de ajuste.

Para cada estación, se verifica si existe en las nuevas tablas de temperatura (`nueva_tabla_tmax` y `nueva_tabla_tmin`). Si la estación está presente, se calcula el valor de ajuste restando la diferencia entre la climatología modelada y observada. Este valor ajustado se utiliza para actualizar los datos en `data_modelo` y se agrega a la fila correspondiente en las tablas de ajuste. Si la estación no está presente, se asigna "x" a la columna correspondiente.

Finalmente, se guardan las tablas de ajuste en archivos CSV y TXT, y se guarda el DataFrame actualizado en un archivo CSV específico para la fecha actual en la carpeta `WRF_Ajust_Clim` dentro de `carpeta_modelo`. Si `data_modelo` es `None`, se imprime un mensaje indicando que no se pudieron obtener los datos correctamente. En resumen, este código realiza ajustes en las temperaturas pronosticadas y guarda los resultados en tablas de ajuste y archivos específicos.

In [401]:
# Verificar si data_modelo es None
if data_modelo is not None:
    for date, file in zip(file_date, data_modelo):

        fecha = datetime.strptime(date, "%Y%m%d")
        
        # Ruta completa del archivo que contendrá la tabla de ajuste
        ruta_ajuste_tmax = os.path.join(carpeta, f"tabla_ajuste_tmax_{fecha.year}.csv")
        ruta_ajuste_tmin = os.path.join(carpeta, f"tabla_ajuste_tmin_{fecha.year}.csv")

        # Si el archivo no existe, se crea la tabla vacía y se descargan y procesan los datos
        if os.path.exists(ruta_ajuste_tmax) and os.path.exists(ruta_ajuste_tmin):
            # Continuar con el procesamiento
            tabla_ajuste_tmax = pd.read_csv(ruta_ajuste_tmax)
            tabla_ajuste_tmin = pd.read_csv(ruta_ajuste_tmin)
        else:
            # Si el archivo existe, cargar la tabla y procesar el día siguiente
            tabla_ajuste_tmax = pd.DataFrame(columns=['FECHA'] + [str(col) for col in file['Estacion']])
            tabla_ajuste_tmin = pd.DataFrame(columns=['FECHA'] + [str(col) for col in file['Estacion']])
        
        # Convertir fecha_siguiente_str a np.int64 para hacer la comparación correctamente
        fecha_siguiente_int = np.int64(date)

        # Verificar si la fecha ya existe en la tabla
        if (tabla_ajuste_tmax["FECHA"] == fecha_siguiente_int).any() or (tabla_ajuste_tmin["FECHA"] == fecha_siguiente_int).any():
            # Si la fecha ya existe, obtener el índice de la fila existente
            idx_fecha_existente_tmax = tabla_ajuste_tmax.loc[tabla_ajuste_tmax["FECHA"] == fecha_siguiente_int].index[0]
            idx_fecha_existente_tmin = tabla_ajuste_tmin.loc[tabla_ajuste_tmin["FECHA"] == fecha_siguiente_int].index[0]

            # Eliminar la fila existente a partir del índice
            if idx_fecha_existente_tmax is not None:
                tabla_ajuste_tmax = tabla_ajuste_tmax.drop(idx_fecha_existente_tmax)

            if idx_fecha_existente_tmin is not None:
                tabla_ajuste_tmin = tabla_ajuste_tmin.drop(idx_fecha_existente_tmin)

        # Crear una nueva fila con la fecha actual
        tabla_tmax_fila = {col: "" for col in tabla_ajuste_tmax.columns}
        tabla_tmax_fila["FECHA"] = date

        tabla_tmin_fila = {col: "" for col in tabla_ajuste_tmin.columns}
        tabla_tmin_fila["FECHA"] = date

        for estacion in tabla_ajuste_tmax.columns[1:]:
            if estacion in Clim_Resta_Tmax.columns:
                # Filtrar según el valor de 'mes'
                valor_resta_tmax = Clim_Resta_Tmax.loc[Clim_Resta_Tmax["mes"] == fecha.month, estacion].values[0]

                # Verificar si existen filas que coincidan con la estación en data_modelo
                filas_estacion = file.loc[file["Estacion"].astype(str).str.strip() == estacion.strip()]

                # Verificar que la fila para la 'estación' en temperatura pronosticada no esté vacía
                if not filas_estacion.empty:
                    # Obtener el valor de temperatura pronosticada para la estación y el mes correspondiente
                    valor_modelo_tmax = filas_estacion["Tmax"].iloc[0]

                    # Ajustar el valor de temperatura pronosticada restando la diferencia entre la climatología modelada y observada
                    nuevo_valor_tmax = valor_modelo_tmax - valor_resta_tmax

                    # Actualizar el valor en data_modelo con el ajuste realizado
                    file.loc[filas_estacion.index, "Tmax"] = nuevo_valor_tmax

                    # Añadir el nuevo valor ajustado a la nueva fila
                    tabla_tmax_fila[estacion] = f"{nuevo_valor_tmax}({valor_modelo_tmax})"

                else:
                    # Si no hay valor para la estación en data_modelo, asignar "x"
                    tabla_tmax_fila[estacion] = "x"
            else:
                print(f"La estación '{estacion}' no existe en la nueva tabla de tmax en la fecha {date}.")
                tabla_tmax_fila[estacion] = "-"

        for estacion in tabla_ajuste_tmin.columns[1:]:
            if estacion in Clim_Resta_Tmin.columns:
                # Filtrar según el valor de 'mes'
                valor_resta_tmin = Clim_Resta_Tmin.loc[Clim_Resta_Tmin["mes"] == fecha.month, estacion].values[0]

                # Verificar si existen filas que coincidan con la estación en data_modelo
                filas_estacion = file.loc[file["Estacion"].astype(str).str.strip() == estacion.strip()]

                # Verificar que la fila para la 'estación' en temperatura pronosticada no esté vacía
                if not filas_estacion.empty:
                    # Obtener el valor de temperatura pronosticada para la estación y el mes correspondiente
                    valor_modelo_tmin = filas_estacion["Tmin"].iloc[0]

                    # Ajustar el valor de temperatura pronosticada restando la diferencia entre la climatología modelada y observada
                    nuevo_valor_tmin = valor_modelo_tmin - valor_resta_tmin

                    # Actualizar el valor en data_modelo con el ajuste realizado
                    file.loc[filas_estacion.index, "Tmin"] = nuevo_valor_tmin

                    # Añadir el nuevo valor ajustado a la nueva fila
                    tabla_tmin_fila[estacion] = f"{nuevo_valor_tmin}({valor_modelo_tmin})"

                else:
                    # Si no hay valor para la estación en data_modelo, asignar "x"
                    tabla_tmin_fila[estacion] = "x"
            else:
                print(f"La estación '{estacion}' no existe en la nueva tabla de tmin en la fecha {date}.")
                tabla_tmin_fila[estacion] = "-"

        # Convert the dictionaries into Pandas DataFrames
        tabla_tmax_fila = pd.DataFrame.from_dict(tabla_tmax_fila, orient='index').T
        tabla_tmin_fila = pd.DataFrame.from_dict(tabla_tmin_fila, orient='index').T

        # Concatenate the DataFrames
        tabla_ajuste_tmax = pd.concat([tabla_ajuste_tmax, tabla_tmax_fila], ignore_index=True)
        tabla_ajuste_tmin = pd.concat([tabla_ajuste_tmin, tabla_tmin_fila], ignore_index=True)

        # Guardar la tabla ajustada en un archivo CSV y TXT
        tabla_ajuste_tmax.to_csv(os.path.join(carpeta, f"tabla_ajuste_tmax_{fecha.year}.csv"), index=False)
        tabla_ajuste_tmax.to_csv(os.path.join(carpeta, f"tabla_ajuste_tmax_{fecha.year}.txt"), index=False)
        
        tabla_ajuste_tmin.to_csv(os.path.join(carpeta, f"tabla_ajuste_tmin_{fecha.year}.csv"), index=False)
        tabla_ajuste_tmin.to_csv(os.path.join(carpeta, f"tabla_ajuste_tmin_{fecha.year}.txt"), index=False)
        
        # Guardar el DataFrame en un archivo CSV con el nombre generado
        file.to_csv(os.path.join(carpeta_modelo, 'WRF_Ajust_Clim', f"{date}.temperatura.csv"), index=False)
        file.to_csv(os.path.join(carpeta_modelo, 'WRF_Ajust_Clim', f"{date}.temperatura.txt"), index=False)

else:
    print("No se pudieron obtener los datos correctamente.")

La estación '69699' no existe en la nueva tabla de tmax en la fecha 20231204.
La estación '69709' no existe en la nueva tabla de tmax en la fecha 20231204.
La estación '69711' no existe en la nueva tabla de tmax en la fecha 20231204.
La estación '69715' no existe en la nueva tabla de tmax en la fecha 20231204.
La estación '69717' no existe en la nueva tabla de tmax en la fecha 20231204.
La estación '69721' no existe en la nueva tabla de tmax en la fecha 20231204.
La estación '69723' no existe en la nueva tabla de tmax en la fecha 20231204.
La estación '69729' no existe en la nueva tabla de tmax en la fecha 20231204.
La estación '69731' no existe en la nueva tabla de tmax en la fecha 20231204.
La estación '69735' no existe en la nueva tabla de tmax en la fecha 20231204.
La estación '71015' no existe en la nueva tabla de tmax en la fecha 20231204.
La estación '72159' no existe en la nueva tabla de tmax en la fecha 20231204.
La estación '72163' no existe en la nueva tabla de tmax en la fe

La estación '73153' no existe en la nueva tabla de tmin en la fecha 20231205.
La estación '74051' no existe en la nueva tabla de tmin en la fecha 20231205.
La estación '74071' no existe en la nueva tabla de tmin en la fecha 20231205.
La estación '74081' no existe en la nueva tabla de tmin en la fecha 20231205.
La estación '76059' no existe en la nueva tabla de tmin en la fecha 20231205.
La estación '76061' no existe en la nueva tabla de tmin en la fecha 20231205.
La estación '78035' no existe en la nueva tabla de tmin en la fecha 20231205.
La estación '82013' no existe en la nueva tabla de tmin en la fecha 20231205.
La estación '82015' no existe en la nueva tabla de tmin en la fecha 20231205.
La estación '84251' no existe en la nueva tabla de tmin en la fecha 20231205.
La estación '94013' no existe en la nueva tabla de tmin en la fecha 20231205.
La estación '96003' no existe en la nueva tabla de tmin en la fecha 20231205.
La estación '98091' no existe en la nueva tabla de tmin en la fe

## Ajuste de datos con el promedio de la resta de los 3 días anteriores

### Crear o Abrir Tabla de Promedios de Resta de los 3 días anteriores

Se genera una fecha para el dia siguiente (`fecha`), una fecha anterior (hoy) (`fecha_anterior`), y se extrae el año correspondiente (`ano`). Se define el nombre de la tabla que se creará o abrirá para contener el promedio de la diferencia entre pronóstico y observado. Luego, se verifica si la carpeta especificada existe; si no, se crea. Se construye la ruta completa del archivo de la tabla, y si el archivo no existe, se inicializa la tabla vacía con la columna 'Estacion' del primer conjunto de datos en `data_modelo`. En caso de que `data_modelo` sea `None`, se imprime un mensaje de error. Si el archivo ya existe, los datos se cargan desde el archivo CSV. Se realiza el procedimiento para temperatura máxima y temperatura minima.

In [402]:
# Obtener la fecha actual y la fecha anterior
fecha = datetime.now().date() + timedelta(days=1)
fecha_anterior = datetime.now().date()

# Obtener el año de la fecha actual
ano = fecha.year

# Nombre de las tablas que contendrán el promedio de la diferencia (pronóstico - observado)
nombre_tabla_tmax = f"tabla_promedios_tmax_{ano}"
nombre_tabla_tmin = f"tabla_promedios_tmin_{ano}"

# Rutas completas de los archivos CSV que contendrán las tablas
ruta_archivo_tmax = os.path.join(carpeta, f"{nombre_tabla_tmax}.csv")
ruta_archivo_tmin = os.path.join(carpeta, f"{nombre_tabla_tmin}.csv")

# Si los archivos existen, cargar los datos de las tablas
if os.path.exists(ruta_archivo_tmax) and os.path.exists(ruta_archivo_tmin):
    tabla_promedios_tmax = pd.read_csv(ruta_archivo_tmax)
    tabla_promedios_tmin = pd.read_csv(ruta_archivo_tmin)
else:
    # Si los archivos no existen y hay datos_modelo, crear tablas con la columna "Estacion"
    if data_modelo is not None:
        tabla_promedios_tmax = data_modelo[0][["Estacion"]]
        tabla_promedios_tmin = data_modelo[0][["Estacion"]]
    else:
        print("No se pudieron obtener los datos correctamente.")

### Verificar si hay datos del pronóstico ajustados para los 3 dían anteriores

Se especifica la ruta de dos carpetas. La primera carpeta, "carpeta_wrf_ajust," contiene datos ajustados del modelo WRF, mientras que la segunda carpeta, "carpeta_obs_crudo," almacena datos observados.

La ejecución del código está condicionada a la existencia y validez de la carpeta "carpeta_wrf_ajust." Si la carpeta existe y es un directorio válido, el programa procede a verificar la cantidad de archivos en dicha carpeta. Si hay al menos 8 archivos, se inicia un proceso de carga de datos.

Para cada fecha en la lista de fechas de pronóstico ("file_date"), el código construye las rutas completas de los archivos de pronóstico y observados en las carpetas correspondientes. Si ambos archivos existen, se leen en pandas DataFrames y se agregan a las listas "data_modelo" y "data_observada" respectivamente. También, la fecha se agrega a la lista "dates."

En caso de que un archivo no se encuentre en las carpetas especificadas, se imprime un mensaje indicando la ausencia del archivo para esa fecha.

Si la carpeta "carpeta_wrf_ajust" tiene menos de 8 archivos, se imprime un mensaje indicando la insuficiencia de archivos y se informa que no se cargarán datos ajustados del WRF. Si la carpeta no existe o no es un directorio válido, se emiten mensajes informando de esta situación y también se indica que no se cargarán archivos ajustados del WRF.

Finalmente, si las variables "file_date" o "data_modelo" no contienen datos, se imprime un mensaje indicando que una o ambas tablas están vacías.

In [403]:
# Especificar la ruta de la carpeta que contiene los datos ajustados del modelo WRF
carpeta_wrf_ajust = f"{carpeta}/Modelo/WRF_Ajustado/"
carpeta_obs_crudo = f"{carpeta}/Observado/Crudo/"

if file_date and data_modelo:
    # Verificar si la carpeta existe y es un directorio válido
    if os.path.exists(carpeta_wrf_ajust) and os.path.isdir(carpeta_wrf_ajust):
        # Obtener la lista de archivos en la carpeta
        archivos_en_carpeta = os.listdir(carpeta_wrf_ajust)

        # Contar la cantidad de archivos en la carpeta
        cantidad_archivos = len(archivos_en_carpeta)

        # Verificar si hay 8 o más archivos en la carpeta
        if cantidad_archivos >= 8:
            print(f"La carpeta tiene 8 o más archivos. Cantidad de archivos: {cantidad_archivos}")
            data_modelo, data_observada, dates = [], [], []

            # Iterar sobre las fechas de pronóstico
            for date in file_date:
                # Crear la ruta completa del archivo de pronóstico en el directorio local
                ruta_wrf_ajust = os.path.join(carpeta_wrf_ajust, f"{date}.temperatura.csv")
                ruta_obs_crudo = os.path.join(carpeta_obs_crudo, f"{date}.temperatura.csv")

                # Verificar si el archivo existe en la carpetas
                if os.path.exists(ruta_wrf_ajust) and os.path.exists(ruta_obs_crudo):
                    # Leer el archivo CSV y agregar los datos al conjunto de datos 'data_modelo'
                    file_wrf = pd.read_csv(ruta_wrf_ajust)
                    data_modelo.append(file_wrf)
                    # Leer el archivo CSV y agregar los datos al conjunto de datos 'data_observada'
                    file_obs = pd.read_csv(ruta_obs_crudo)
                    data_observada.append(file_obs)
                    # Agregar las fechas de datos a 'dates'
                    dates.append(date)
                else:
                    # Imprimir un mensaje si el archivo no se encuentra en la carpeta de datos ajustados del WRF
                    print(f'El archivo para el {date} no se encuentra en la carpeta de datos del WRF ajustados o observados')
        else:
            # Imprimir un mensaje si la carpeta tiene menos de 8 archivos
            print(f"La carpeta tiene menos de 8 archivos. Cantidad de archivos: {cantidad_archivos}")
            print("No se cargan archivos del WRF ya ajustados")
    else:
        # Imprimir un mensaje si la carpeta no existe o no es un directorio válido
        print("La carpeta no existe o no es un directorio válido.")
        print("No se cargan archivos del WRF ya ajustados")
else:
    print("Alguna de las tablas o ambas estan vacias")

La carpeta tiene 8 o más archivos. Cantidad de archivos: 28


### Obtener la resta (pronostico menos observado) de los 3 días anteriores

En este fragmento de código, se crea una lista llamada `tablas_diferencias` destinada a almacenar los resultados de las diferencias entre los datos pronosticados y observados de temperaturas máximas y mínimas. Se verifica la presencia de datos tanto en `data_modelo` como en `data_observada`.

En caso de que ambas listas contengan datos, se utiliza un bucle `for` para recorrer las listas de DataFrames correspondientes a los datos pronosticados (`data_modelo`) y observados (`data_observada`). Para cada iteración, se realiza una combinación de los DataFrames mediante la columna "Estacion" utilizando el método `merge`, y se calculan las diferencias entre las temperaturas pronosticadas y observadas para cada día. Estas diferencias se almacenan en las columnas "Diferencia_Tmax" y "Diferencia_Tmin".

Posteriormente, se filtran las columnas para seleccionar únicamente las necesarias, que son "Estacion", "Diferencia_Tmax" y "Diferencia_Tmin". Se eliminan las filas con valores NaN. La tabla resultante se agrega a la lista `tablas_diferencias`.

En el caso de que alguna de las listas (`data_modelo` o `data_observada`) esté vacía, se imprime un mensaje indicando que al menos una de las tablas está vacía o ambas.

En resumen, este código calcula y almacena las diferencias entre las temperaturas pronosticadas y observadas para cada estación y día, y organiza estos resultados en una lista llamada `tablas_diferencias`.

In [404]:
# Crear una lista para almacenar los resultados (temperatura pronosticada menos temperatura observada)
tablas_diferencias = []

if data_modelo and data_observada:
    # Recorrer las listas de DataFrames (pro y obs) para calcular las diferencias (pro - obs) para cada día
    for i in range(len(data_modelo)):

        # Realizar una combinación "left" entre pro y obs en base a la columna "Estacion"
        diferencias = data_modelo[i].merge(data_observada[i], on="Estacion", how="left", suffixes=("_pro", "_obs"))

        # Calcular las diferencias (pro - obs) y guardarlo en la columna 'Diferencia_Tmax' y 'Diferencia_Tmin'
        diferencias["Diferencia_Tmax"] = diferencias["Tmax_pro"] - diferencias["Tmax_obs"]
        diferencias["Diferencia_Tmin"] = diferencias["Tmin_pro"] - diferencias["Tmin_obs"]

        # Filtrar para seleccionar solo las columnas necesarias
        diferencias = diferencias[["Estacion", "Diferencia_Tmax", "Diferencia_Tmin"]]
        
        # Eliminar filas con valores NaN en la columna 'Resta'
        diferencias = diferencias.dropna(subset=["Diferencia_Tmax", "Diferencia_Tmin"])

        # Agregar la tabla diferencias a la lista tablas_diferencias
        tablas_diferencias.append(diferencias)
else:
    print("Alguna de las tablas (datos_modelo y data_observada) o ambas estan vacias")## Pronostico menos Observado

### Obtener el promedio de la resta de los 3 días anteriores

Se utiliza `pd.concat()` para concatenar las tablas de diferencias presentes en la lista `tablas_diferencias` en una sola tabla llamada `tabla_concatenada`. Se calcula el promedio de las diferencias agrupando por estación para las temperaturas máximas (`tabla_promedio_tmax`) y mínimas (`tabla_promedio_tmin`). El método `groupby` se utiliza para agrupar por la columna "Estacion", y luego se aplica la función `mean()` para calcular el promedio.

La columna de diferencias para temperaturas máximas se renombra como la fecha en formato texto (`text`) en `tabla_promedio_tmax`, y para temperaturas mínimas en `tabla_promedio_tmin`. Se verifica si la columna correspondiente a la fecha ya existe en las tablas `tabla_promedios_tmax` y `tabla_promedios_tmin`. En caso afirmativo, se elimina la columna existente.

Se fusionan las tablas `tabla_promedios_tmax` y `tabla_promedios_tmin` con las nuevas tablas `tabla_promedio_tmax` y `tabla_promedio_tmin`, respectivamente. Se utiliza el método `merge` con la columna "Estacion" como clave y el modo "outer" para incluir todas las estaciones. Finalmente, se guardan las tablas resultantes (`tabla_promedios_tmax` y `tabla_promedios_tmin`) en archivos CSV y TXT en la carpeta especificada (`carpeta`).

En caso de que la lista `tablas_diferencias` esté vacía, se imprime un mensaje indicando que la tabla de diferencias está vacía.

In [405]:
from datetime import datetime, date, timedelta  # Importamos clases relacionadas con fechas y horas.

if tablas_diferencias:
    # Concatenar las tres tablas de tablas_diferencias en una sola tabla
    tabla_concatenada = pd.concat(tablas_diferencias)

    # Calcular el promedio agrupando por Estación
    tabla_promedio_tmax = tabla_concatenada.groupby("Estacion")["Diferencia_Tmax"].mean().reset_index()
    tabla_promedio_tmin = tabla_concatenada.groupby("Estacion")["Diferencia_Tmin"].mean().reset_index()

    # Obtener la fecha actual en formato texto y renombrar la columna 'Resta' de tabla_promedio
    text = fecha.strftime("%Y%m%d")
    tabla_promedio_tmax = tabla_promedio_tmax.rename(columns={"Diferencia_Tmax": f"{text}"})
    tabla_promedio_tmin = tabla_promedio_tmin.rename(columns={"Diferencia_Tmin": f"{text}"})

    # Verificar si la columna ya existe en tabla_promedios_tmax, tabla_promedios_tmin
    if f"{text}" in tabla_promedios_tmax.columns:
        # Si existe, eliminarla
        tabla_promedios_tmax.drop(columns=[f"{text}"], inplace=True)

    if f"{text}" in tabla_promedios_tmin.columns:
        # Si existe la columna existe, eliminarla
        tabla_promedios_tmin.drop(columns=[f"{text}"], inplace=True)

    # Fusionar tabla_promedios_tmax, tabla_promedios_tmin con tabla_promedio_tmax, tabla_promedio_tmin
    tabla_promedios_tmax = tabla_promedios_tmax.merge(tabla_promedio_tmax, on="Estacion", how="outer")
    tabla_promedios_tmin = tabla_promedios_tmin.merge(tabla_promedio_tmin, on="Estacion", how="outer")

    # Guardar la tabla_promedios_tmax, tabla_promedios_tmin en un archivo CSV y TXT
    tabla_promedios_tmax.to_csv(f"{carpeta}/tabla_promedios_tmax_{ano}.csv", index=False)
    tabla_promedios_tmax.to_csv(os.path.join(carpeta, f"tabla_promedios_tmax_{ano}.txt"), index=False)

    tabla_promedios_tmin.to_csv(f"{carpeta}/tabla_promedios_tmin_{ano}.csv", index=False)
    tabla_promedios_tmin.to_csv(os.path.join(carpeta, f"tabla_promedios_tmin_{ano}.txt"), index=False)
else:
    print("Tabla diferencias está vacía")

### Cargar pronóstico de mañana

Se construye un enlace completo (`url_modelo_mañana`) para el archivo CSV del pronóstico de temperatura del día siguiente. El enlace se forma utilizando la base del enlace (`enlace_base_mod`), la fecha actual (`fecha_anterior`) y la fecha siguiente (`fecha`), todas formateadas en el formato adecuado.

Se verifica la existencia del archivo en el enlace. Si el estado de la respuesta (`response.status_code`) es 200, significa que el archivo existe y se puede proceder con la descarga. Se descarga el archivo y se procesa para asegurar la integridad de los datos. Se comprueba si hay valores negativos o de tipo string en las columnas de temperaturas pronosticadas. Si se encuentran valores negativos o de tipo string, se imprime un mensaje indicando que no se carga el archivo.

Se sustituyen los valores de tipo string por NaN en las columnas de temperaturas (`Tmax` y `Tmin`).Se guarda el DataFrame resultante en archivos CSV y TXT en la carpeta especificada (`carpeta_modelo`, en la subcarpeta `WRF_Crudo`). El nombre del archivo se genera utilizando la fecha actual (`fecha`) formateada correctamente.

Si el estado de la respuesta no es 200, se imprime un mensaje indicando que hay archivos faltantes para la fecha actual.

In [406]:
# Formar el enlace completo para el archivo CSV con la fecha actual y siguiente¿
url_modelo_mañana = f"{enlace_base_mod}{fecha_anterior.strftime('%Y%m%d')}/{fecha.strftime('%Y%m%d')}.temperatura.csv"

# Verificar si el archivo de temperatura observada existe antes de descargarlo
response = requests.get(url_modelo_mañana)

if response.status_code == 200:
    csv_data = response.content.decode('utf-8')
    df = pd.read_csv(io.StringIO(csv_data), header=None, names=COLUMN_NAMES)

    negative_values = (df[COLUMN_NAMES[1:]] < 0).any(axis=0)
    str_values = df[COLUMN_NAMES[1:]].applymap(lambda x: isinstance(x, str)).any(axis=0)

    if (negative_values.any() or str_values.any()):
        print(f"Valores de temperatura pronosticada para {fecha} son negativos o del tipo str. No se carga el archivo.")
    else:
        # Sustituir los valores str por NaN en las columnas
        df[["Tmax", "Tmin"]] = df[["Tmax", "Tmin"]].apply(pd.to_numeric, errors='coerce')
        
        file = df[['Estacion', "Tmax", "Tmin"]]

        # Guardar el DataFrame en un archivo CSV con el nombre generado
        file.to_csv(os.path.join(carpeta_modelo, 'WRF_Crudo', f"{fecha.strftime('%Y%m%d')}.temperatura.csv"), index=False)
        file.to_csv(os.path.join(carpeta_modelo, 'WRF_Crudo', f"{fecha.strftime('%Y%m%d')}.temperatura.txt"), index=False)
else:
    print(f"Hay archivos faltantes para la fecha {fecha.strftime('%Y%m%d')}")

### Restar al pronóstico de mañana el promedio de la resta de los 3 días previos

Se verifica la existencia de tres variables (`tabla_promedio_tmax`, `tabla_promedio_tmin`, y `file`). Se utiliza la estructura condicional `if "variable" in locals()` para verificar si cada una de las tres variables (`tabla_promedio_tmax`, `tabla_promedio_tmin`, y `file`) existe en el entorno actual.

Se verifica si las variables existen y no son nulas (`None`). Luego, se comprueba si las variables no están vacías mediante la condición `if not variable.empty`. Si todas las verificaciones son exitosas, se procede a realizar un ajuste al pronóstico de temperatura para el día siguiente. Se realiza la resta del pronóstico de mañana con el promedio de los tres días previos para las temperaturas máximas y mínimas.

Se crea un nuevo DataFrame llamado `Pronostico` que contiene la estación y las columnas ajustadas de temperaturas máximas (`Tmax`) y mínimas (`Tmin`). Se guardan los resultados del ajuste en archivos CSV y TXT en la carpeta especificada (`carpeta`). El nombre del archivo se genera utilizando la fecha actual (`fecha`) formateada correctamente.

Si alguna de las variables (`tabla_promedio_tmax`, `tabla_promedio_tmin`, y `file`) está vacía o no está definida, se imprime un mensaje indicando la situación correspondiente. Si alguna de las variables no existe en el entorno actual, se imprime un mensaje indicando la situación correspondiente.

In [407]:
if "tabla_promedio_tmax" in locals() and "tabla_promedio_tmin" in locals() and "file" in locals():
    if tabla_promedio_tmax is not None and tabla_promedio_tmin is not None and file is not None:
        if not tabla_promedio_tmax.empty and not tabla_promedio_tmin.empty and not file.empty:
            # Verificar si hay NaN en los DataFrames
            if tabla_promedio_tmax.isnull().values.any() or tabla_promedio_tmin.isnull().values.any() or file.isnull().values.any():
                print("Alguno de los DataFrames contiene valores NaN.")
            else:
                # Restarle al pronóstico de mañana el promedio de los 3 días previos
                Pronostico_tmax = file[["Estacion", "Tmax"]]
                Pronostico_tmin = file[["Estacion", "Tmin"]]

                merged_tmax = pd.merge(Pronostico_tmax, tabla_promedio_tmax, on='Estacion', how='left')
                merged_tmin = pd.merge(Pronostico_tmin, tabla_promedio_tmin, on='Estacion', how='left')

                Pronostico = pd.DataFrame()
                Pronostico["Estacion"] = merged_tmax["Estacion"]

                Pronostico["Tmax"] = merged_tmax["Tmax"] - merged_tmax[f"{text}"].fillna(0)
                Pronostico["Tmin"] = merged_tmin["Tmin"] - merged_tmin[f"{text}"].fillna(0)

                # Guarda el DataFrame del ajuste del pronóstico de mañana en archivos CSV y TXT
                Pronostico.to_csv(os.path.join(carpeta_modelo, 'WRF_Ajustado', f"{text}.temperatura.csv"), index=False)
                Pronostico.to_csv(os.path.join(carpeta_modelo, 'WRF_Ajustado', f"{text}.temperatura.txt"), index=False)
        else:
            print("Alguna de las tablas (tabla_promedio_tmax, tabla_promedio_tmin y file) o ambas están vacías")
    else:
        print("Alguna de las tablas (tabla_promedio_tmax, tabla_promedio_tmin y file) no está definida")
else:
    print("Alguna de las tablas (tabla_promedio_tmax, tabla_promedio_tmin y file) no existe en el entorno actual.")

**Hay estaciones que aveces no tienen un datos, puedo no ajustar el pronostico si el valor es NaN?
Ejemplo: Estacion 69699 ahorita tiene el sensor de Temp en mantenimiento, si hago el ajuste el pronostico ajustado será NaN para esas estaciones, cuando use ese pronostico para ajustar otro día seguirá dando NaN**

In [408]:
Pronostico

,Estacion,Tmax,Tmin
0,69633,28.418466,20.716702
1,69647,26.445390,21.639424
2,69679,29.729584,22.144937
3,69699,26.070000,22.050000
4,69701,29.746071,21.201305
5,69709,29.399662,21.199941
6,69711,30.062821,24.147281
7,69713,30.389170,20.085436
8,69715,29.509849,20.935085
9,69717,29.161191,21.083081


In [409]:
# Configurar opciones para mostrar todas las filas y columnas sin truncar
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

# Restaurar las opciones predeterminadas si es necesario
#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')